## HW 9
## Hongbo Liu

1. Get HTML
Get the content of the page into R.

2. Get the info box
On the right side of the page is a box of structured content, called an info box. Wikipedia has many types of such info boxes to provide content comparably for a group of articles of the same class (e.g. the Members of the U.S. senate, Fortune 500 companies, Crime Syndicates etc.)

a) Find the CSS class of the infobox.

Through copying the "selector" from the https://en.wikipedia.org/wiki/Mitch_McConnell, I know the CSS class of the infobox is "infobox.vcard"

b) Extract the part of the HTML document that contains the infobox using the CSS information.

In [190]:
import lxml
from lxml import html 
import requests


In [191]:
link="https://en.wikipedia.org/wiki/Mitch_McConnell"
wiki=requests.get(link)
wiki_doc=html.fromstring(wiki.content)

In [192]:
from lxml.cssselect import CSSSelector


In [193]:
Info=wiki_doc.cssselect(".infobox.vcard> tbody >tr")

In [194]:
Info[3][0][0].text

'Senate Minority Leader'

3. Make a data frame

a) Parse the infobox table HTML you obtained above into a data frame.

b) Name the columns of the table you obtain key and value. So, in the example for Mitch McConnell, "Whip" would be the key, and the content information (i.e. the value) is "John Thune".

In [195]:
#mw-content-text > div.mw-parser-output > table.infobox.vcard > tbody > tr:nth-child(8) > th

In [196]:
key=wiki_doc.xpath("//*[@id='mw-content-text']/div[1]/table[1]/tbody/tr/th/descendant-or-self::*/text()")

In [197]:
value=wiki_doc.xpath("//*[@id='mw-content-text']/div[1]/table[1]/tbody/tr/td/descendant::*/text()")

In [198]:
import pandas as pd

In [199]:
value=value[0:51]


In [200]:
df=pd.DataFrame({"key":key,"value":value})

In [201]:
df.head()

,key,value
0,Mitch McConnell,"Official portrait, 2016"
1,Senate Minority Leader,Incumbent
2,Whip,Assumed office
3,Preceded by,John Thune
4,Whip,Chuck Schumer


c) Filter the data frame (and rename variables if necessary) to the "Full name", "Political Party", and "Children". Use this selection of variables for all subsequent questions.

In [202]:
pp=wiki_doc.xpath("//tbody/tr/*[contains(text(),'Political party')]/parent::tr/td/a/text()")

In [203]:
c=wiki_doc.xpath("//tbody/tr/*[contains(text(),'Children')]/parent::tr/td/text()")

In [204]:
n=wiki_doc.xpath("//tbody/tr/th/descendant-or-self::*/text()")[0]

In [205]:
data=[["Political party",pp],["Children",c],["Full name",n]]
df = pd.DataFrame(data, columns=['key', 'value'])
df

,key,value
0,Political party,[Republican]
1,Children,[3]
2,Full name,Mitch McConnell


4. Make a function

a) Use the code above to make a function called get_wiki_info that uses a single input url (a Wikipedia URL) and outputs the data frame of the format above. There is no need to account for exceptions (e.g. no info box on the page; page does not exist etc.) - we will only use members of the U.S. Senate for this exercise.

In [206]:
def get_wiki_info(url):
    wiki=requests.get(url)
    wiki_doc=html.fromstring(wiki.content)
    pp=wiki_doc.xpath("//tbody/tr/*[contains(text(),'Political party')]/parent::tr/td/a/text()")
    c=wiki_doc.xpath("//tbody/tr/*[contains(text(),'Children')]/parent::tr/td/text()")
    n=wiki_doc.xpath("//tbody/tr/th/descendant-or-self::*/text()")[0]
    if not c:
        c="NA"
    data=[["Political party",pp],["Children",c],["Full name",n]]
    df = pd.DataFrame(data, columns=['key', 'value'])
    
    return df

In [207]:
get_wiki_info(url="https://en.wikipedia.org/wiki/Mitch_McConnell")

,key,value
0,Political party,[Republican]
1,Children,[3]
2,Full name,Mitch McConnell


b) Show how your function works on the following two URLs:

In [208]:
get_wiki_info("https://en.wikipedia.org/wiki/Tammy_Duckworth")

,key,value
0,Political party,[Democratic]
1,Children,[2]
2,Full name,Tammy Duckworth


In [209]:
get_wiki_info("https://en.wikipedia.org/wiki/Susan_Collins")

,key,value
0,Political party,[Republican]
1,Children,NA
2,Full name,Susan Collins
